# This code was used to calculate the centroids for polygons

In [1]:
import pandas as pd
from Classes.Database import Database
from shapely import geometry
import re

### The first function calculates the centroid
### The second function was used to create the sql insertion queries (although we can use sqlalchemy)

In [2]:
def create_centroid(val):
    if val is None:
        return None
    points = []
    for i in val.split('],'):
        points.append(re.findall(r"[-+]?\d*\.\d+|\d+", i))
        points[-1][0] = float(points[-1][0])
        points[-1][1] = float(points[-1][1])
    myplgn = geometry.Polygon(points)
    return re.findall(r"[-+]?\d*\.\d+|\d+", myplgn.centroid.wkt)
def getTweetsInsertQuery(tweets):
    query = ""
    for tweet in tweets:
        temp = "('{0}', '{1}', '{2}', '{3}', {4}, {5}, {6}, {7}, '{8}', {9}, {10}, {11})".format(tweet[0],
                                     tweet[1],
                                     tweet[2],
                                     tweet[3],
                                     ensureDBNull(tweet[4]),
                                     ensureDBNull(tweet[5]),
                                     tweet[10],
                                     tweet[11],
                                     tweet[6],
                                     ensureDBNull(tweet[7]),
                                     ensureDBNull(tweet[8]),
                                     ensureDBNull(tweet[9]))
        query = query + ",\n" + temp
#         query = "{0},\n{1}".format(query, temp) 
                   
    query = 'REPLACE INTO TWEETS VALUES\n' + query[2:] + ';'
#     query = 'REPLACE INTO TWEETS VALUES\n{0};'.format(query)
    return query
def ensureDBNull(cell):
    return 'NULL' if cell is None else ("'" + cell + "'")

In [ ]:
allTweets = pd.read_sql("SELECT * FROM tweetCoords", Database().myDB)

In [ ]:
longs = []
latds = []
kk = 0
tweets = allTweets.copy()
cords = tweets['coordinates']
for i in tweets['location_coordinates']:
    if cords[kk] is not None or i is None:
        lat, lon = re.findall(r"[-+]?\d*\.\d+|\d+", cords[kk])
    else:
        lon, lat = create_centroid(i)
    latds.append(lat)
    longs.append(lon)
    kk+=1
tweets['longitude'] = longs
tweets['latitudes'] = latds